## Download and store OSM network data.
### Packages

- Import standard Python packages.
- Import local modules for route shape matching.

In [2]:
# Import standard packages
import os
import sys

# Import local module
sys.path.append('../src/')
import route_shape_matching.rm_utilities as rmu

### Directory & Parameters
- Set directory to read the geospatial files.
- Set or import parameters from local modules such as coordinate reference system. 

In [3]:
# Directory
data_dir = os.path.normpath(r'..\\data')
os.makedirs(os.path.join(data_dir, 'processed', 'osm_graph'), exist_ok = True)
output_dir = os.path.join(data_dir, 'processed', 'osm_graph')

# Parameters
CRS = rmu.CRS
CRS_METER = rmu.CRS_METER

# GIS route line geometries 
gis_lines_geojson = os.path.join(data_dir,
                                 'external',
                                 'gis_bsrt_line',
                                 'gis_bsrt_line.geojson')

### Read data

This data should have the geometries for the universal set of routes for which we want to extract the OSM network.

In [3]:
gis_routes = rmu.read_all_gis_route_lines(gis_lines_geojson, CRS)

### Create unified polygon
- Use a GeoDataFrame to create the boundary of a unified polygon. 
- Define a buffer threshold to include additional area around the actual boundary to capture nearly connected nodes in the street network. 

In [4]:
polygon_geometry = rmu.get_unified_polygon(gis_routes, CRS, buffer_threshold = 0.005)

### Download OSM data
- Run this function to download and create a graph object for the given boundary. 
- The function downloads the filtered network excluding some link types that are not valid for transit route shape matching.
- If some parts of the data or the entire data are available in the cache from a previous query, the function reads the data from that cache folder bypassing the download step.
- By default, this function prints logs in the console. However, this can be controled using the `log_console` argument.
- Using the cache, it takes ~ 7 mins to create and simplyfy the graph. Downloading will take additional time and it may vary as the API requests are sent at random interval to avoid overflow. 

In [5]:
G = rmu.download_osm_for_transit(polygon_geometry, log_console = True)

### Export OSM Graph and Shapefiles
- Export the graph object of OSM street network in `.graphml` format.
- If `export_nodes` or `export_edges` is set to `True`, the function also exports relevant GeoDataFrames in `.geojson` format. 
- It usually takes 6 mins to export all files.

In [7]:
rmu.export_osm_graph_shapefiles(graph_obj = G, 
                                output_path = output_dir, 
                                export_nodes = True, 
                                export_edges = True)